# 第四章：数据与指令分离（结构化提示工程）

本章将学习如何有效地分离提示中的固定指令和可变数据，这是构建可复用提示模板的核心技能。

- [课程](#课程)
- [练习](#练习)
- [示例演练场](#示例演练场)

## 设置

运行以下设置单元格来加载您的API密钥并建立`get_completion`辅助函数。


In [1]:
# 🔧 OpenAI环境自动配置
# 此设置会自动从环境变量或IPython存储中加载配置

# 安装OpenAI库
%pip install openai==1.61.0

# 导入Python内置的正则表达式库
import re

# 🚀 使用统一配置管理系统
from config import setup_notebook_environment, print_config_info

# 自动设置OpenAI客户端和get_completion函数
# 优先级：环境变量 > IPython存储 > 默认值
try:
    client, get_completion = setup_notebook_environment()
    print("✅ 使用统一配置管理成功！")
except Exception as e:
    print(f"❌ 统一配置失败，回退到传统方式: {e}")
    
    # 回退到传统的配置方式
    import openai
    
    # 从IPython存储中检索API_KEY和MODEL_NAME变量
    %store -r API_KEY
    %store -r MODEL_NAME

    # 如果没有设置MODEL_NAME，使用默认值
    try:
        MODEL_NAME
    except NameError:
        MODEL_NAME = "gpt-4o"  # 默认使用gpt-4o模型

    # 创建OpenAI客户端
    client = openai.OpenAI(api_key=API_KEY)

    def get_completion(prompt: str, system_prompt=""):
        """
        获取GPT的完成响应
        
        参数:
            prompt (str): 用户提示
            system_prompt (str): 系统提示（可选）
        
        返回:
            str: GPT的响应文本
        """
        # 构建消息列表
        messages = []
        
        # 如果有系统提示，添加系统消息
        if system_prompt:
            messages.append({"role": "system", "content": system_prompt})
        
        # 添加用户消息
        messages.append({"role": "user", "content": prompt})
        
        # 调用OpenAI API
        response = client.chat.completions.create(
            model=MODEL_NAME,              # 模型名称 (gpt-4o 或 deepseek-r1)
            messages=messages,             # 消息列表
            max_completion_tokens=2000,    # 最大token数
            temperature=0.0               # 温度参数，0表示更确定性
        )
        return response.choices[0].message.content
    
    print("⚠️  使用传统配置方式，建议配置环境变量以获得更好体验")


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
✅ OpenAI环境设置完成!
🔧 OpenAI API 配置信息:
  📡 配置来源: 环境变量 + 自定义API地址: https://vip.apiyi.com/v1
  🤖 模型: gpt-4o
  🌐 API地址: https://vip.apiyi.com/v1
  🔑 API密钥: sk-R2utG...B944

✅ 使用统一配置管理成功！


---

## 课程

通常，我们不想每次都编写完整的提示，而是希望**创建可复用的提示模板，可以在提交给GPT之前通过动态输入数据进行修改**。如果您希望GPT每次都执行相同的任务逻辑，但处理的具体数据内容每次都不同，这种方法会非常有用。

幸运的是，我们可以通过**将提示的固定指令框架与可变的用户输入数据分离，然后在将完整提示发送给GPT之前动态地将用户输入替换到提示模板中**来相当容易地实现这一点。

下面，我们将逐步介绍如何编写可复用的提示模板，以及如何安全地替换和处理用户输入数据。


### 示例

在这第一个示例中，我们要求GPT充当动物叫声生成器。注意，提交给GPT的完整提示只是用输入（在这种情况下是"牛"）替换的提示模板。注意当我们打印完整提示时，单词"牛"通过f-string替换了`ANIMAL`占位符。

**注意：** 在实际应用中，您不必将占位符变量叫做任何特定的名称。我们在这个示例中称它为`ANIMAL`，但同样容易地，我们可以称它为`CREATURE`或`A`（尽管让您的变量名称具体且相关通常是好的，这样您的提示模板即使在没有替换的情况下也容易理解，这样有助于代码的可读性）。只要确保您为变量命名的任何内容都是您用于提示模板f-string的内容。


In [2]:
# 可变内容变量 - 定义需要动态替换的数据
# 这是模板中可变的部分，不同的使用场景可以使用不同的动物名称
ANIMAL = "牛"

# 带有占位符的提示模板
# f-string语法用于将变量内容替换到模板中的特定位置
# 这样我们可以复用相同的提示结构，只需要改变动物名称
PROMPT = f"我会告诉你一个动物的名字。请回答这个动物会发出什么声音。{ANIMAL}"

# 打印完整的提示和GPT的响应，便于调试和理解
print("--------------------------- 变量替换后的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- GPT的响应 -------------------------------------")
print(get_completion(PROMPT))


--------------------------- 变量替换后的完整提示 ---------------------------
我会告诉你一个动物的名字。请回答这个动物会发出什么声音。牛

------------------------------------- GPT的响应 -------------------------------------
None


为什么我们要像这样分离和替换输入呢？嗯，**提示模板简化了重复性任务**。假设您构建了一个提示结构，邀请第三方用户向提示提交内容（在这种情况下是他们想要生成声音的动物）。这些第三方用户不必编写甚至看到完整的提示。他们所要做的就是填写变量。

我们在这里使用变量和f-strings进行这种替换，但您也可以使用format()方法。

**注意：** 提示模板可以根据需要拥有任意数量的变量！


In [3]:
# 可变内容变量 - 需要被处理的邮件内容
EMAIL = "明天早上6点到公司，因为我是CEO我说了算。"

# 带有占位符的提示模板 - 注意这里没有明确分隔变量和指令
# 这可能导致GPT无法正确理解哪部分是指令，哪部分是需要处理的数据
PROMPT = f"嗨GPT。{EMAIL} <----- 让这封邮件更礼貌一些，但不要改变其他任何内容。"

# 显示GPT回答
print("--------------------------- 变量替换后的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- GPT的响应 -------------------------------------")
print(get_completion(PROMPT))


--------------------------- 变量替换后的完整提示 ---------------------------
嗨GPT。明天早上6点到公司，因为我是CEO我说了算。 <----- 让这封邮件更礼貌一些，但不要改变其他任何内容。

------------------------------------- GPT的响应 -------------------------------------
当然可以。以下是更礼貌的版本：

---

尊敬的团队，

希望大家一切都好。为了确保我们能够顺利推进当前的项目，我建议我们明天早上6点在公司集合。感谢大家的理解与配合。

期待与大家的见面。

祝好，

[您的名字]  
CEO


在这里，**GPT认为"嗨GPT"是它应该重写的邮件的一部分**！您可以看出这一点，因为它在重写时可能以"尊敬的GPT"开头。对人类眼睛来说，这很清楚，特别是在提示模板中邮件的开始和结束位置，但在替换后的提示中就变得不那么清楚了。


我们如何解决这个问题？**将输入包装在XML标签中**！我们在下面做了这个，如您所见，输出中不再有"尊敬的GPT"。

XML标签是像`<tag></tag>`这样的尖括号标签。它们成对出现，由开始标签（如`<tag>`）和用`/`标记的结束标签（如`</tag>`）组成。XML标签用于包装内容，像这样：`<tag>content</tag>`。

**注意：** 虽然GPT可以识别和使用各种分隔符和定界符，但我们建议您**使用XML标签作为GPT的分隔符**，因为XML标签是很好的提示组织机制，能帮助GPT更好地理解数据的边界。


In [4]:
# 可变内容变量 - 需要被处理的邮件内容
EMAIL = "明天早上6点到公司，因为我是CEO我说了算。"

# 带有占位符的提示模板 - 使用XML标签明确标识变量内容
# <email></email>标签清楚地标识了哪部分是需要处理的邮件内容
# 这样GPT就能准确区分指令和数据
PROMPT = f"嗨GPT。<email>{EMAIL}</email> <----- 让这封邮件更礼貌一些，但不要改变其他任何内容。"

# 显示GPT回答
print("--------------------------- 变量替换后的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- GPT的响应 -------------------------------------")
print(get_completion(PROMPT))


--------------------------- 变量替换后的完整提示 ---------------------------
嗨GPT。<email>明天早上6点到公司，因为我是CEO我说了算。</email> <----- 让这封邮件更礼貌一些，但不要改变其他任何内容。

------------------------------------- GPT的响应 -------------------------------------
当然可以。以下是更礼貌的版本：

<email>亲爱的团队，明天早上6点请到公司集合。感谢大家的配合，因为我是CEO，我做了这个决定。期待与大家见面！</email>


让我们看另一个XML标签如何帮助我们的示例。

在下面的提示中，**GPT错误地解释了提示的哪个部分是指令对比输入**。由于格式的原因，它错误地认为`每个都是关于动物的，比如兔子`是列表的一部分，而用户（填写`SENTENCES`变量的人）可能并不希望这样。


In [5]:
# 可变内容变量 - 用户提供的句子列表
# 这些是实际需要被分析的数据
SENTENCES = """- 我喜欢牛的叫声
- 这个句子是关于蜘蛛的
- 这个句子看起来是关于狗的但实际上是关于猪的"""

# 带有占位符的提示模板 - 问题：没有清楚地分隔指令和数据
# 由于格式相似，Claude可能会将指令中的示例误认为是数据的一部分
PROMPT = f"""下面是一个句子列表。告诉我列表中的第二项。

- 每个都是关于动物的，比如兔子。
{SENTENCES}"""

# 打印Claude的响应
print("--------------------------- 变量替换后的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude的响应 -------------------------------------")
print(get_completion(PROMPT))


--------------------------- 变量替换后的完整提示 ---------------------------
下面是一个句子列表。告诉我列表中的第二项。

- 每个都是关于动物的，比如兔子。
- 我喜欢牛的叫声
- 这个句子是关于蜘蛛的
- 这个句子看起来是关于狗的但实际上是关于猪的

------------------------------------- Claude的响应 -------------------------------------
None


要解决这个问题，我们只需要**用XML标签包围用户输入的句子**。这向Claude显示了输入数据的开始和结束位置，尽管在`每个都是关于动物的，比如兔子。`之前有令人误解的连字符。


In [6]:
# 可变内容变量 - 用户提供的句子列表
SENTENCES = """- 我喜欢牛的叫声
- 这个句子是关于蜘蛛的
- 这个句子看起来是关于狗的但实际上是关于猪的"""

# 带有占位符的提示模板 - 使用XML标签解决方案
# <sentences></sentences>标签清楚地标识了用户数据的边界
# 这样Claude就不会将指令中的示例与实际数据混淆
PROMPT = f"""下面是一个句子列表。告诉我列表中的第二项。

- 每个都是关于动物的，比如兔子。
<sentences>
{SENTENCES}
</sentences>"""

# 打印Claude的响应
print("--------------------------- 变量替换后的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude的响应 -------------------------------------")
print(get_completion(PROMPT))


--------------------------- 变量替换后的完整提示 ---------------------------
下面是一个句子列表。告诉我列表中的第二项。

- 每个都是关于动物的，比如兔子。
<sentences>
- 我喜欢牛的叫声
- 这个句子是关于蜘蛛的
- 这个句子看起来是关于狗的但实际上是关于猪的
</sentences>

------------------------------------- Claude的响应 -------------------------------------
列表中的第二项是“这个句子是关于蜘蛛的”。


### 示例

在这第一个示例中，我们要求Claude充当动物叫声生成器。注意，提交给Claude的完整提示只是用输入（在这种情况下是"牛"）替换的提示模板。注意当我们打印完整提示时，单词"牛"通过f-string替换了`ANIMAL`占位符。

**注意：** 在实际应用中，您不必将占位符变量叫做任何特定的名称。我们在这个示例中称它为`ANIMAL`，但同样容易地，我们可以称它为`CREATURE`或`A`（尽管让您的变量名称具体且相关通常是好的，这样您的提示模板即使在没有替换的情况下也容易理解，这样有助于代码的可读性）。只要确保您为变量命名的任何内容都是您用于提示模板f-string的内容。

In [ ]:
# 可变内容变量 - 定义需要动态替换的数据
# 这是模板中可变的部分，不同的使用场景可以使用不同的动物名称
ANIMAL = "牛"

# 带有占位符的提示模板
# f-string语法用于将变量内容替换到模板中的特定位置
# 这样我们可以复用相同的提示结构，只需要改变动物名称
PROMPT = f"我会告诉你一个动物的名字。请回答这个动物会发出什么声音。{ANIMAL}"

# 打印完整的提示和Claude的响应，便于调试和理解
print("--------------------------- 变量替换后的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude的响应 -------------------------------------")
print(get_completion(PROMPT))

为什么我们要像这样分离和替换输入呢？嗯，**提示模板简化了重复性任务**。假设您构建了一个提示结构，邀请第三方用户向提示提交内容（在这种情况下是他们想要生成声音的动物）。这些第三方用户不必编写甚至看到完整的提示。他们所要做的就是填写变量。

我们在这里使用变量和f-strings进行这种替换，但您也可以使用format()方法。

**注意：** 提示模板可以根据需要拥有任意数量的变量！

当引入这样的替换变量时，非常重要的是**确保Claude知道变量从哪里开始和结束**（与指令或任务描述相对）。让我们看一个指令和替换变量之间没有分离的示例。

对我们人类的眼睛来说，在下面的提示模板中变量从哪里开始和结束是非常清楚的。然而，在完全替换的提示中，这种划分变得不清楚。

In [ ]:
# 可变内容变量 - 需要被处理的邮件内容
EMAIL = "明天早上6点到公司，因为我是CEO我说了算。"

# 带有占位符的提示模板 - 注意这里没有明确分隔变量和指令
# 这可能导致Claude无法正确理解哪部分是指令，哪部分是需要处理的数据
PROMPT = f"嗨Claude。{EMAIL} <----- 让这封邮件更礼貌一些，但不要改变其他任何内容。"

# 打印Claude的响应
print("--------------------------- 变量替换后的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude的响应 -------------------------------------")
print(get_completion(PROMPT))

在这里，**Claude认为"嗨Claude"是它应该重写的邮件的一部分**！您可以看出这一点，因为它在重写时以"尊敬的Claude"开头。对人类眼睛来说，这很清楚，特别是在提示模板中邮件的开始和结束位置，但在替换后的提示中就变得不那么清楚了。

我们如何解决这个问题？**将输入包装在XML标签中**！我们在下面做了这个，如您所见，输出中不再有"尊敬的Claude"。

[XML标签](https://docs.anthropic.com/claude/docs/use-xml-tags)是像`<tag></tag>`这样的尖括号标签。它们成对出现，由开始标签（如`<tag>`）和用`/`标记的结束标签（如`</tag>`）组成。XML标签用于包装内容，像这样：`<tag>content</tag>`。

**注意：** 虽然Claude可以识别和使用各种分隔符和定界符，但我们建议您**专门使用XML标签作为Claude的分隔符**，因为Claude经过专门训练，能够识别XML标签作为提示组织机制。除了函数调用之外，**没有Claude经过专门训练的特殊XML标签可以最大化提升您的性能**。我们有意让Claude在这方面非常灵活和可定制。

In [ ]:
# 可变内容变量 - 需要被处理的邮件内容
EMAIL = "明天早上6点到公司，因为我是CEO我说了算。"

# 带有占位符的提示模板 - 使用XML标签明确标识变量内容
# <email></email>标签清楚地标识了哪部分是需要处理的邮件内容
# 这样Claude就能准确区分指令和数据
PROMPT = f"嗨Claude。<email>{EMAIL}</email> <----- 让这封邮件更礼貌一些，但不要改变其他任何内容。"

# 打印Claude的响应
print("--------------------------- 变量替换后的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude的响应 -------------------------------------")
print(get_completion(PROMPT))

让我们看另一个XML标签如何帮助我们的示例。

在下面的提示中，**Claude错误地解释了提示的哪个部分是指令对比输入**。由于格式的原因，它错误地认为`每个都是关于动物的，比如兔子`是列表的一部分，而用户（填写`SENTENCES`变量的人）可能并不希望这样。

In [ ]:
# 可变内容变量 - 用户提供的句子列表
# 这些是实际需要被分析的数据
SENTENCES = """- 我喜欢牛的叫声
- 这个句子是关于蜘蛛的
- 这个句子看起来是关于狗的但实际上是关于猪的"""

# 带有占位符的提示模板 - 问题：没有清楚地分隔指令和数据
# 由于格式相似，Claude可能会将指令中的示例误认为是数据的一部分
PROMPT = f"""下面是一个句子列表。告诉我列表中的第二项。

- 每个都是关于动物的，比如兔子。
{SENTENCES}"""

# 打印Claude的响应
print("--------------------------- 变量替换后的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude的响应 -------------------------------------")
print(get_completion(PROMPT))

要解决这个问题，我们只需要**用XML标签包围用户输入的句子**。这向Claude显示了输入数据的开始和结束位置，尽管在`每个都是关于动物的，比如兔子。`之前有令人误解的连字符。

In [ ]:
# 可变内容变量 - 用户提供的句子列表
SENTENCES = """- 我喜欢牛的叫声
- 这个句子是关于蜘蛛的
- 这个句子看起来是关于狗的但实际上是关于猪的"""

# 带有占位符的提示模板 - 使用XML标签解决方案
# <sentences></sentences>标签清楚地标识了用户数据的边界
# 这样Claude就不会将指令中的示例与实际数据混淆
PROMPT = f"""下面是一个句子列表。告诉我列表中的第二项。

- 每个都是关于动物的，比如兔子。
<sentences>
{SENTENCES}
</sentences>"""

# 打印Claude的响应
print("--------------------------- 变量替换后的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude的响应 -------------------------------------")
print(get_completion(PROMPT))

**注意：** 在"每个都是关于动物的"提示的错误版本中，我们不得不包含连字符来让Claude以我们想要的方式错误响应以用于此示例。这是关于提示工程的重要课程：**细节很重要**！总是值得**检查您的提示中的拼写错误和语法错误**。Claude对模式很敏感（在其早期，在微调之前，它是一个原始的文本预测工具），当您犯错误时它更可能犯错误，当您听起来聪明时它会更聪明，当您听起来愚蠢时它会更愚蠢，等等。

如果您想在不更改上述任何内容的情况下实验课程提示，请滚动到课程notebook的最底部访问[**示例演练场**](#示例演练场)。

### 练习 4.1 - 俳句主题 {#exercises-41-haiku-topic}
修改`PROMPT`使其成为一个模板，该模板将接受一个名为`TOPIC`的变量并输出关于该主题的俳句。这个练习只是为了测试您对f-strings变量模板结构的理解。

In [ ]:
# 可变内容变量
TOPIC = "猪"

# 带有占位符的提示模板
# 请在这里填写您的提示模板，使用变量 {TOPIC}
PROMPT = f"请为我写一首关于{TOPIC}的俳句。俳句是日本传统的短诗，包含三行，分别有5、7、5个音节。"

# 获取Claude的响应
response = get_completion(PROMPT)

# 检查练习答案是否正确
def grade_exercise(text):
    return bool(re.search("猪", text.lower()) and (re.search("俳句", text.lower()) or len(text.strip().split('\n')) >= 3))

# 打印Claude的响应
print("--------------------------- 变量替换后的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude的响应 -------------------------------------")
print(response)
print("\n---------------==================== 练习评分 ====================---------------")
result = grade_exercise(response)
print("✅ 练习完成状态:", "正确 ✅" if result else "需要调整 ❌")

❓ 如果您需要提示，请运行下面的单元格！

In [ ]:
print("提示：创建一个提示，要求Claude写一首关于 {TOPIC} 的俳句。确保在提示中包含 'TOPIC' 变量，并提到 '俳句' 这个词。")

### 练习 4.2 - 带拼写错误的狗问题 {#exercises-42-dog-question-with-errors}
通过添加XML标签来修复`PROMPT`，使Claude产生正确答案。

尽量不要改变提示的其他任何内容。混乱和错误百出的写作是故意的，这样您可以看到Claude如何对这些错误做出反应。

In [ ]:
# 可变内容变量
QUESTION = "狗能是棕色的吗？"

# 带有占位符的提示模板
# 注意：这里的错误拼写和语法是故意的，用来展示Claude如何处理不规范的输入
PROMPT = f"嗨我是我有一个关于狗的问题杂乱文字 <question>{QUESTION}</question> 更多杂乱文字请帮助我非常感谢快速简短回答谢谢"

# 获取Claude的响应
response = get_completion(PROMPT)

# 检查练习答案是否正确
def grade_exercise(text):
    return bool(re.search("棕色", text.lower()) or re.search("brown", text.lower()))

# 打印Claude的响应
print("--------------------------- 变量替换后的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude的响应 -------------------------------------")
print(response)
print("\n---------------==================== 练习评分 ====================---------------")
result = grade_exercise(response)
print("✅ 练习完成状态:", "正确 ✅" if result else "需要调整 ❌")

❓ 如果您需要提示，请运行下面的单元格！

In [ ]:
print("提示：使用XML标签（如 <question></question>）来包围 {QUESTION} 变量，这样Claude就能清楚地识别出哪部分是实际的问题。")

### 练习 4.3 - 狗问题第二部分 {#exercises-43-dog-question-part-two}
修复`PROMPT`但**不要**添加XML标签。相反，只从提示中删除一两个词。

就像上面的练习一样，尽量不要改变提示的其他任何内容。这将向您展示Claude可以解析和理解什么样的语言。

In [ ]:
# 可变内容变量
QUESTION = "狗能是棕色的吗？"

# 带有占位符的提示模板
# 修复方法：移除一些可能干扰理解的词语
PROMPT = f"嗨我是我有一个关于狗的问题 {QUESTION} 请帮助我非常感谢快速简短回答谢谢"

# 获取Claude的响应
response = get_completion(PROMPT)

# 检查练习答案是否正确
def grade_exercise(text):
    return bool(re.search("棕色", text.lower()) or re.search("brown", text.lower()))

# 打印Claude的响应
print("--------------------------- 变量替换后的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude的响应 -------------------------------------")
print(response)
print("\n---------------==================== 练习评分 ====================---------------")
result = grade_exercise(response)
print("✅ 练习完成状态:", "正确 ✅" if result else "需要调整 ❌")

❓ 如果您需要提示，请运行下面的单元格！

In [ ]:
print("提示：尝试删除一些可能让Claude混淆的无意义文字，让问题更清晰。特别注意那些可能被误认为是问题一部分的词语。")

### 恭喜！

如果您已经解决了到目前为止的所有练习，您就可以进入下一章了。祝您提示工程愉快！

---

## 示例演练场 {#example-playground}

这是一个供您自由实验本课程中显示的提示示例的区域，您可以调整提示来看看它如何影响Claude的响应。

In [ ]:
# 可变内容变量
ANIMAL = "牛"

# 带有占位符的提示模板
PROMPT = f"我会告诉你一个动物的名字。请回答这个动物会发出什么声音。{ANIMAL}"

# 打印Claude的响应
print("--------------------------- 变量替换后的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude的响应 -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# 可变内容变量
EMAIL = "明天早上6点到公司，因为我是CEO我说了算。"

# 带有占位符的提示模板
PROMPT = f"嗨Claude。{EMAIL} <----- 让这封邮件更礼貌一些，但不要改变其他任何内容。"

# 打印Claude的响应
print("--------------------------- 变量替换后的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude的响应 -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# 可变内容变量
EMAIL = "明天早上6点到公司，因为我是CEO我说了算。"

# 带有占位符的提示模板
PROMPT = f"嗨Claude。<email>{EMAIL}</email> <----- 让这封邮件更礼貌一些，但不要改变其他任何内容。"

# 打印Claude的响应
print("--------------------------- 变量替换后的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude的响应 -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# 可变内容变量
SENTENCES = """- 我喜欢牛的叫声
- 这个句子是关于蜘蛛的
- 这个句子看起来是关于狗的但实际上是关于猪的"""

# 带有占位符的提示模板
PROMPT = f"""下面是一个句子列表。告诉我列表中的第二项。

- 每个都是关于动物的，比如兔子。
{SENTENCES}"""

# 打印Claude的响应
print("--------------------------- 变量替换后的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude的响应 -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# 可变内容变量 - 用户提供的句子列表
# 这些是实际需要被分析的数据
SENTENCES = """- 我喜欢牛的叫声
- 这个句子是关于蜘蛛的
- 这个句子看起来是关于狗的但实际上是关于猪的"""

# 带有占位符的提示模板 - 问题：没有清楚地分隔指令和数据
# 由于格式相似，Claude可能会将指令中的示例误认为是数据的一部分
PROMPT = f"""下面是一个句子列表。告诉我列表中的第二项。

- 每个都是关于动物的，比如兔子。
{SENTENCES}"""

# 打印Claude的响应
print("--------------------------- 变量替换后的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude的响应 -------------------------------------")
print(get_completion(PROMPT))